In [1]:
from rbc.mapd import RemoteMapD

In [2]:
omnisci = RemoteMapD()  # RemoteMapD(user=, password=, host=, port=, dbname=)

In [3]:
omnisci.sql_execute('drop table if exists mytable')
omnisci.sql_execute('create table if not exists mytable (i INT[]);');
for i in range(5):
    omnisci.sql_execute('insert into mytable values (ARRAY[1,2,3,'+str(i)+'])');
list(omnisci.sql_execute('select * from mytable')[1])

[([1, 2, 3, 0],),
 ([1, 2, 3, 1],),
 ([1, 2, 3, 2],),
 ([1, 2, 3, 3],),
 ([1, 2, 3, 4],)]

In [4]:
@omnisci('i32(i32[])')
def sum_i32(x):
    s = 0
    for i in range(len(x)):
        s += x[i]
    return s

In [5]:
descr, result = omnisci.sql_execute(
    'select i, sum_i32(i) from mytable')

list(result)

[([1, 2, 3, 0], 6),
 ([1, 2, 3, 1], 7),
 ([1, 2, 3, 2], 8),
 ([1, 2, 3, 3], 9),
 ([1, 2, 3, 4], 10)]

In [6]:
@omnisci('i32(i32[])')
def len_i32(x):
    return len(x)
descr, result = omnisci.sql_execute(
    'select i, len_i32(i) from mytable')

list(result)

[([1, 2, 3, 0], 4),
 ([1, 2, 3, 1], 4),
 ([1, 2, 3, 2], 4),
 ([1, 2, 3, 3], 4),
 ([1, 2, 3, 4], 4)]

In [7]:
print(len_i32.get_IR())

; ModuleID = 'rbc.irtools.compile_to_IR'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

; Function Attrs: norecurse nounwind readonly
define i32 @len_i32({ i32*, i64, i8 }* nocapture readonly %.1) local_unnamed_addr #0 {
entry:
  %.17.i.i = getelementptr { i32*, i64, i8 }, { i32*, i64, i8 }* %.1, i64 0, i32 1
  %.18.i.i = load i64, i64* %.17.i.i, align 8, !noalias !0
  %.38.i = trunc i64 %.18.i.i to i32
  ret i32 %.38.i
}

attributes #0 = { norecurse nounwind readonly }

!0 = !{!1, !3}
!1 = distinct !{!1, !2, !"_ZN3rbc13omnisci_array14mapd_array_len12$3clocals$3e16$3clambda$3e$242E22$28STRUCT__iPLBK$29$2a: %retptr"}
!2 = distinct !{!2, !"_ZN3rbc13omnisci_array14mapd_array_len12$3clocals$3e16$3clambda$3e$242E22$28STRUCT__iPLBK$29$2a"}
!3 = distinct !{!3, !4, !"_ZN8__main__11len_i32$246E22$28STRUCT__iPLBK$29$2a: %retptr"}
!4 = distinct !{!4, !"_ZN8__main__11len_i32$246E22$28STRUCT__iPLBK$29$2a"}



In [8]:
Array_getSize_i32_ir = '''\
define dso_local i64 @Array_getSize_i32(%struct.Array*) align 2 {
  %2 = alloca %struct.Array*, align 8
  store %struct.Array* %0, %struct.Array** %2, align 8
  %3 = load %struct.Array*, %struct.Array** %2, align 8
  %4 = getelementptr inbounds %struct.Array, %struct.Array* %3, i32 0, i32 1
  %5 = load i64, i64* %4, align 8
  ret i64 %5
}
'''
print(Array_getSize_i32_ir)

define dso_local i64 @Array_getSize_i32(%struct.Array*) align 2 {
  %2 = alloca %struct.Array*, align 8
  store %struct.Array* %0, %struct.Array** %2, align 8
  %3 = load %struct.Array*, %struct.Array** %2, align 8
  %4 = getelementptr inbounds %struct.Array, %struct.Array* %3, i32 0, i32 1
  %5 = load i64, i64* %4, align 8
  ret i64 %5
}

